<a href="https://colab.research.google.com/github/LambdaTheda/DS-Unit-2-Kaggle-Challenge/blob/master/730p_Sun_Apr_19_Unit_2_after_u2s2m4_DS_Sprint_Challenge_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

_Lambda School Data Science, Unit 2_
 
# Sprint Challenge: Predict Steph Curry's shots 🏀

For your Sprint Challenge, you'll use a dataset with all Steph Curry's NBA field goal attempts. (Regular season and playoff games, from October 28, 2009, through June 5, 2019.) 

You'll predict whether each shot was made, using information about the shot and the game. This is hard to predict! Try to get above 60% accuracy. The dataset was collected with the [nba_api](https://github.com/swar/nba_api) Python library.

In [0]:
%%capture
import sys

if 'google.colab' in sys.modules:
    # Install packages in Colab
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*

In [3]:
# Read data
import pandas as pd
url = 'https://drive.google.com/uc?export=download&id=1fL7KPyxgGYfQDsuJoBWHIWwCAf-HTFpX'
df = pd.read_csv(url)

# Check data shape
assert df.shape == (13958, 20)

#Explore data
df.shape # assertion above checks
df.describe()



,game_id,game_event_id,period,minutes_remaining,seconds_remaining,shot_distance,loc_x,loc_y,shot_made_flag,scoremargin_before_shot
count,1.395800e+04,13958.000000,13958.000000,13958.000000,13958.000000,13958.000000,13958.000000,13958.000000,13958.000000,13958.000000
mean,2.442837e+07,270.438458,2.412380,4.727540,28.506376,17.600373,-0.554162,131.257988,0.472919,1.615561
std,7.226620e+06,169.927170,1.125828,3.331646,17.597701,10.295807,124.721869,102.666562,0.499284,10.127139
min,2.090002e+07,2.000000,1.000000,0.000000,0.000000,0.000000,-250.000000,-41.000000,0.000000,-39.000000
25%,2.120091e+07,109.000000,1.000000,2.000000,13.000000,8.000000,-96.000000,23.000000,0.000000,-4.000000
50%,2.150026e+07,275.500000,2.000000,4.000000,29.000000,22.000000,1.000000,141.000000,0.000000,1.000000
75%,2.170096e+07,398.000000,3.000000,7.000000,44.000000,25.000000,95.000000,219.000000,1.000000,8.000000
max,4.180040e+07,752.000000,6.000000,11.000000,59.000000,83.000000,247.000000,811.000000,1.000000,43.000000


In [4]:
df.head()

,game_id,game_event_id,player_name,period,minutes_remaining,seconds_remaining,action_type,shot_type,shot_zone_basic,shot_zone_area,shot_zone_range,shot_distance,loc_x,loc_y,shot_made_flag,game_date,htm,vtm,season_type,scoremargin_before_shot
0,20900015,4,Stephen Curry,1,11,25,Jump Shot,3PT Field Goal,Above the Break 3,Right Side Center(RC),24+ ft.,26,99,249,0,2009-10-28,GSW,HOU,Regular Season,2.0
1,20900015,17,Stephen Curry,1,9,31,Step Back Jump shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,18,-122,145,1,2009-10-28,GSW,HOU,Regular Season,0.0
2,20900015,53,Stephen Curry,1,6,2,Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),8-16 ft.,14,-60,129,0,2009-10-28,GSW,HOU,Regular Season,-4.0
3,20900015,141,Stephen Curry,2,9,49,Jump Shot,2PT Field Goal,Mid-Range,Left Side(L),16-24 ft.,19,-172,82,0,2009-10-28,GSW,HOU,Regular Season,-4.0
4,20900015,249,Stephen Curry,2,2,19,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,16,-68,148,0,2009-10-28,GSW,HOU,Regular Season,0.0


In [5]:
# Number of unique Visiting Teams 
df['vtm'].nunique() # returns 32

32

In [6]:
# List of 32 Visiting Teams 
df['vtm'].unique()

array(['HOU', 'GSW', 'MEM', 'LAC', 'MIN', 'POR', 'LAL', 'IND', 'ORL',
       'SAS', 'WAS', 'PHX', 'BOS', 'SAC', 'CLE', 'MIA', 'MIL', 'CHI',
       'DEN', 'NJN', 'NOH', 'CHA', 'OKC', 'DAL', 'UTA', 'ATL', 'PHI',
       'DET', 'TOR', 'NYK', 'BKN', 'NOP'], dtype=object)

To demonstrate mastery on your Sprint Challenge, do all the required, numbered instructions in this notebook.

To earn a score of "3", also do all the stretch goals.

You are permitted and encouraged to do as much data exploration as you want.

**1. Begin with baselines for classification.** Your target to predict is `shot_made_flag`. What is your baseline accuracy, if you guessed the majority class for every prediction?

**2. Hold out your test set.** Use the 2018-19 season to test. NBA seasons begin in October and end in June. You'll know you've split the data correctly when your test set has 1,709 observations.

**3. Engineer new feature.** Engineer at least **1** new feature, from this list, or your own idea.
- **Homecourt Advantage**: Is the home team (`htm`) the Golden State Warriors (`GSW`) ?
- **Opponent**: Who is the other team playing the Golden State Warriors?
- **Seconds remaining in the period**: Combine minutes remaining with seconds remaining, to get the total number of seconds remaining in the period.
- **Seconds remaining in the game**: Combine period, and seconds remaining in the period, to get the total number of seconds remaining in the game. A basketball game has 4 periods, each 12 minutes long.
- **Made previous shot**: Was Steph Curry's previous shot successful?

**4. Decide how to validate** your model. Choose one of the following options. Any of these options are good. You are not graded on which you choose.
- **Train/validate/test split: train on the 2009-10 season through 2016-17 season, validate with the 2017-18 season.** You'll know you've split the data correctly when your train set has 11,081 observations, and your validation set has 1,168 observations.
- **Train/validate/test split: random 80/20%** train/validate split.
- **Cross-validation** with independent test set. You may use any scikit-learn cross-validation method.

**5.** Use a scikit-learn **pipeline** to **encode categoricals** and fit a **Decision Tree** or **Random Forest** model.

**6.** Get your model's **validation accuracy.** (Multiple times if you try multiple iterations.) 

**7.** Get your model's **test accuracy.** (One time, at the end.)


**8.** Given a **confusion matrix** for a hypothetical binary classification model, **calculate accuracy, precision, and recall.**

### Stretch Goals
- Engineer 4+ new features total, either from the list above, or your own ideas.
- Make 2+ visualizations to explore relationships between features and target.
- Optimize 3+ hyperparameters by trying 10+ "candidates" (possible combinations of hyperparameters). You can use `RandomizedSearchCV` or do it manually.
- Get and plot your model's feature importances.



## 1. Begin with baselines for classification. 

>Your target to predict is `shot_made_flag`. What would your baseline accuracy be, if you guessed the majority class for every prediction?

In [7]:
# 1. Begin with baselines for classification. Your target to predict is shot_made_flag. 
#    What is your baseline accuracy, if you guessed the majority class for every prediction?

import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
target = 'shot_made_flag'
''' get test in assignment 2
train, test = train_test_split(df, test_size=.2,  
                              stratify=df[target], random_state=42)
train, val = train_test_split(train, test_size=len(test),  
                              stratify=train[target], random_state=42)
'''
train, val = train_test_split(df, test_size=.2,  
                              stratify=df[target], random_state=42)

# Arrange data into X features matrix and y target vector 
columnsToDrop = [target, 'game_id', 'game_event_id', 'player_name']
X_train = train.drop(columns=columnsToDrop)
y_train = train[target]
X_val = val.drop(columns=columnsToDrop)
y_val = val[target]
#X_test = test.drop(columns=columnsToDrop)


# Make pipeline!
pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(strategy='mean'), 
    RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
)

# Fit on train, score on val
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_val)
print('Validation Accuracy:', accuracy_score(y_val, y_pred))

print('Majority class accuracy: 47.2919%')
print(df[target].value_counts(normalize=True)) # 47% of time Curry gets shot in

Validation Accuracy: 0.6429083094555874
Majority class accuracy: 47.2919%
0    0.527081
1    0.472919
Name: shot_made_flag, dtype: float64


## 2. Hold out your test set.

>Use the 2018-19 season to test. NBA seasons begin in October and end in June. You'll know you've split the data correctly when your test set has 1,709 observations.

In [8]:
test = df[df['game_date'] >= '2018-10-01']
test = test.drop(columns=columnsToDrop)
test.shape

(1709, 16)

## 3. Engineer new feature.

>Engineer at least **1** new feature, from this list, or your own idea.
>
>- **Homecourt Advantage**: Is the home team (`htm`) the Golden State Warriors (`GSW`) ?
>- **Opponent**: Who is the other team playing the Golden State Warriors?
>- **Seconds remaining in the period**: Combine minutes remaining with seconds remaining, to get the total number of seconds remaining in the period.
>- **Seconds remaining in the game**: Combine period, and seconds remaining in the period, to get the total number of seconds remaining in the game. A basketball game has 4 periods, each 12 minutes long.
>- **Made previous shot**: Was Steph Curry's previous shot successful?

    

In [0]:
# adding Homecourt Advantage feature
def isHomeTeam(row):
  if(row['htm'] == 'GSW'):
    return 1
  return 0

df['homecourt_advantage'] = df.apply (lambda row: isHomeTeam(row), axis=1)

## **4. Decide how to validate** your model. 

>Choose one of the following options. Any of these options are good. You are not graded on which you choose.
>
>- **Train/validate/test split: train on the 2009-10 season through 2016-17 season, validate with the 2017-18 season.** You'll know you've split the data correctly when your train set has 11,081 observations, and your validation set has 1,168 observations.
>- **Train/validate/test split: random 80/20%** train/validate split.
>- **Cross-validation** with independent test set. You may use any scikit-learn cross-validation method.

In [0]:
#split
train = df[df['game_date'] >= '2009-10-01']
train = train[train['game_date'] < '2017-08-01']

val = df[df['game_date'] >= '2017-10-01']
val = val[val['game_date'] < '2018-08-01']

test = df[df['game_date'] >= '2018-10-01']

#choose which columns not to look at
columnsToDrop = [target, 'game_id', 'game_event_id', 'player_name']

#split into X and y
X_train = train.drop(columns=columnsToDrop)
y_train = train[target]

X_val = val.drop(columns=columnsToDrop) #Cross-validation with independent test set- doesn't overlap w/training set
y_val = val[target]

X_test = test.drop(columns=columnsToDrop)
y_test = test[target]

## 5. Use a scikit-learn pipeline to encode categoricals and fit a Decision Tree or Random Forest model.

In [0]:
pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(strategy='mean'), 
    RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
)
#fit
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_val)

## 6.Get your model's validation accuracy

> (Multiple times if you try multiple iterations.)

In [12]:
print('Validation Accuracy:', accuracy_score(y_val, y_pred))

Validation Accuracy: 0.5907534246575342


## 7. Get your model's test accuracy

> (One time, at the end.)

In [13]:
test_pred = pipeline.predict(X_test)
print('Test Accuracy:', accuracy_score(y_test, test_pred))

Test Accuracy: 0.6225863077823288


## 8. Given a confusion matrix, calculate accuracy, precision, and recall.

Imagine this is the confusion matrix for a binary classification model. Use the confusion matrix to calculate the model's accuracy, precision, and recall.

<table>
  <tr>
    <td colspan="2" rowspan="2"></td>
    <td colspan="2">Predicted</td>
  </tr>
  <tr>
    <td>Negative</td>
    <td>Positive</td>
  </tr>
  <tr>
    <td rowspan="2">Actual</td>
    <td>Negative</td>
    <td style="border: solid">85</td>
    <td style="border: solid">58</td>
  </tr>
  <tr>
    <td>Positive</td>
    <td style="border: solid">8</td>
    <td style="border: solid"> 36</td>
  </tr>
</table>

### Calculate accuracy 

In [14]:
num_predictions = 85 + 58 + 8 + 36
accurate_predictions = 85 + 36
print('Accuracy:', accurate_predictions/num_predictions)

Accuracy: 0.6470588235294118


### Calculate precision

In [15]:
'''
Since the assignment didn't specify whether we look for
positives or negatives, here are the precisions for both

Precision = actual outcome / predicted outcome
'''
print('Positive Precision:', 36/(58+36))
print('Negative Precision:', 85/(85+8))

Positive Precision: 0.3829787234042553
Negative Precision: 0.9139784946236559


Precision and recall are two extremely important model evaluation metrics. While **precision refers to the percentage of your results which are relevant, recall refers to the percentage of total relevant results correctly classified by your algorithm**.

May 11, 2018
towardsdatascience.com › precision-vs-recall-386cf9f89488

### Calculate recall

In [16]:
#same for recall, including false negatives

print('Positive Recall:', 36/(58+36+8))
print('Negative Recall:', 85/(85+8+58))

Positive Recall: 0.35294117647058826
Negative Recall: 0.5629139072847682


------------------------------------------
#**Stretch Goals:** 
1) Engineer 4+ new features total, either from the list above, or your own ideas.

2) Make 2+ visualizations to explore relationships between features and target.

3) Optimize 3+ hyperparameters by trying 10+ "candidates" (possible combinations of hyperparameters).
   You can use RandomizedSearchCV or do it manually.

4) Get and plot your model's feature importances.

### 1) Engineer 4+ new features total, either from the list above, or your own ideas.


**Feature #1**

In [0]:
# adding 'isScrimmage' column: are the Golden State Warriors playing against themselves?

def isOpponentGSW(row):
  if(row['vtm'] != 'GSW'):
    return 'No'   # if NOT 'GSW'
  return 'Yes'    # if 'GSW', ie, perhaps scrimmaging against teammates

df['isScrimmage?'] = df.apply(lambda row: isOpponentGSW(row), axis=1)

df


**Feature #2**

In [0]:
# Add 'otherTeamName' column: if vtm is NOT GWS, ie, NOT indicative of a scrimmage

def otherTeam(row):
  if (row['isScrimmage?'] == 'No'):
     return row['vtm']
  return 'Scrimmage'

#df['isScrimmage?'] = df.apply(lambda row: isScrimmage(row), axis =1) 

df['otherTeam'] = df.apply(lambda row: otherTeam(row), axis = 1)
 
df


**Feature #3**

In [0]:
# adding 'GLORY SHOT' column, Yes: if 'seconds_remaining' <= 10  or No > 10 

def isGloryShot(row):
  if(row['seconds_remaining'] <= 10):
    return 'Yes'
  return 'No'

df['isGloryShot?'] = df.apply(lambda row: isGloryShot(row), axis = 1) 

df



**Feature #4**

In [0]:
# adding 'HERO SHOT' column, Yes: if 'seconds_remaining' <= 10  AND 'shot_made_flag' == 1  AND 'scoremargin_before_shot'  < 3.
 
def isHeroShot(row):
# if((row['seconds_remaining'] <= 10) and (row['scoremargin_before_shot'] < 3)): # and operator for if statements conds 1 and 2
  if((row['seconds_remaining'] <= 10) & (row['scoremargin_before_shot'] < 3)): # bit-wise operator does & operation.. to select certain rows in a df ..like dereferencing, for #s n chking on rows 
  
  # if()
    return 'Yes'
  return 'No'

df['isHeroShot?'] = df.apply(lambda row: isHeroShot(row), axis = 1)  

df.sample(35)

###2) Make 2+ visualizations to explore relationships between features and target.


In [25]:
df.corr()

,game_id,game_event_id,period,minutes_remaining,seconds_remaining,shot_distance,loc_x,loc_y,shot_made_flag,scoremargin_before_shot,homecourt_advantage
game_id,1.000000,0.058387,0.023757,-0.018650,-0.009982,0.019789,-0.007311,0.016984,-0.017542,-0.022761,0.006558
game_event_id,0.058387,1.000000,0.940855,-0.291271,-0.018433,0.011211,0.037097,0.029789,-0.012720,0.096800,-0.038768
period,0.023757,0.940855,1.000000,-0.071512,0.009031,-0.014679,0.040834,0.002066,-0.005241,0.076163,-0.008206
minutes_remaining,-0.018650,-0.291271,-0.071512,1.000000,0.037336,-0.053699,-0.012034,-0.078049,0.033877,-0.084498,-0.002664
seconds_remaining,-0.009982,-0.018433,0.009031,0.037336,1.000000,-0.064230,-0.010662,-0.068568,0.036618,-0.007952,0.013011
shot_distance,0.019789,0.011211,-0.014679,-0.053699,-0.064230,1.000000,-0.017901,0.822426,-0.138070,0.025137,0.041477
loc_x,-0.007311,0.037097,0.040834,-0.012034,-0.010662,-0.017901,1.000000,0.036513,-0.005824,0.013601,-0.018087
loc_y,0.016984,0.029789,0.002066,-0.078049,-0.068568,0.822426,0.036513,1.000000,-0.130082,0.018910,0.063964
shot_made_flag,-0.017542,-0.012720,-0.005241,0.033877,0.036618,-0.138070,-0.005824,-0.130082,1.000000,-0.003207,0.016526
scoremargin_before_shot,-0.022761,0.096800,0.076163,-0.084498,-0.007952,0.025137,0.013601,0.018910,-0.003207,1.000000,0.139805


In [29]:
corr = df.corr()
print(corr)

                          game_id  ...  homecourt_advantage
game_id                  1.000000  ...             0.006558
game_event_id            0.058387  ...            -0.038768
period                   0.023757  ...            -0.008206
minutes_remaining       -0.018650  ...            -0.002664
seconds_remaining       -0.009982  ...             0.013011
shot_distance            0.019789  ...             0.041477
loc_x                   -0.007311  ...            -0.018087
loc_y                    0.016984  ...             0.063964
shot_made_flag          -0.017542  ...             0.016526
scoremargin_before_shot -0.022761  ...             0.139805
homecourt_advantage      0.006558  ...             1.000000

[11 rows x 11 columns]


In [30]:
corr.style.background_gradient(cmap = 'coolwarm')

,game_id,game_event_id,period,minutes_remaining,seconds_remaining,shot_distance,loc_x,loc_y,shot_made_flag,scoremargin_before_shot,homecourt_advantage
game_id,1.000000,0.058387,0.023757,-0.018650,-0.009982,0.019789,-0.007311,0.016984,-0.017542,-0.022761,0.006558
game_event_id,0.058387,1.000000,0.940855,-0.291271,-0.018433,0.011211,0.037097,0.029789,-0.012720,0.096800,-0.038768
period,0.023757,0.940855,1.000000,-0.071512,0.009031,-0.014679,0.040834,0.002066,-0.005241,0.076163,-0.008206
minutes_remaining,-0.018650,-0.291271,-0.071512,1.000000,0.037336,-0.053699,-0.012034,-0.078049,0.033877,-0.084498,-0.002664
seconds_remaining,-0.009982,-0.018433,0.009031,0.037336,1.000000,-0.064230,-0.010662,-0.068568,0.036618,-0.007952,0.013011
shot_distance,0.019789,0.011211,-0.014679,-0.053699,-0.064230,1.000000,-0.017901,0.822426,-0.138070,0.025137,0.041477
loc_x,-0.007311,0.037097,0.040834,-0.012034,-0.010662,-0.017901,1.000000,0.036513,-0.005824,0.013601,-0.018087
loc_y,0.016984,0.029789,0.002066,-0.078049,-0.068568,0.822426,0.036513,1.000000,-0.130082,0.018910,0.063964
shot_made_flag,-0.017542,-0.012720,-0.005241,0.033877,0.036618,-0.138070,-0.005824,-0.130082,1.000000,-0.003207,0.016526
scoremargin_before_shot,-0.022761,0.096800,0.076163,-0.084498,-0.007952,0.025137,0.013601,0.018910,-0.003207,1.000000,0.139805


In [22]:
#SCATTERPLOT: from https://towardsdatascience.com/better-heatmaps-and-correlation-matrix-plots-in-python-41445d0f2bec

def heatmap(x, y, size):
    fig, ax = plt.subplots()
    
    # Mapping from column names to integer coordinates
    x_labels = [v for v in sorted(x.unique())]
    y_labels = [v for v in sorted(y.unique())]

    x_to_num = {p[1]:p[0] for p in enumerate(x_labels)} 
    y_to_num = {p[1]:p[0] for p in enumerate(y_labels)} 
    
    size_scale = 500

    ax.scatter(
        x=x.map(x_to_num), # Use mapping for x
        y=y.map(y_to_num), # Use mapping for y
        s=size * size_scale, # Vector of square sizes, proportional to size parameter
        marker='s' # Use square as scatterplot marker
    )
    
    # Show column labels on the axes
    ax.set_xticks([x_to_num[v] for v in x_labels])
    ax.set_xticklabels(x_labels, rotation=45, horizontalalignment='right')
    ax.set_yticks([y_to_num[v] for v in y_labels])
    ax.set_yticklabels(y_labels)
    
data = pd.read_csv('https://raw.githubusercontent.com/drazenz/heatmap/master/autos.clean.csv')

columns = ['bore', 'stroke', 'compression-ratio', 'horsepower', 'city-mpg', 'price'] 

corr = data[columns].corr()
corr = pd.melt(corr.reset_index(), id_vars='index') # Unpivot the dataframe, so we can get pair of arrays for x and y

corr.columns = ['x', 'y', 'value']

heatmap(
    x=corr['x'],
    y=corr['y'],
    size=corr['value'].abs()
)

NameError: ignored

3) Optimize 3+ hyperparameters by trying 10+ "candidates" (possible combinations of hyperparameters). You can use RandomizedSearchCV or do it manually.